<a href="https://www.kaggle.com/code/nofal82/food-11-usingtransferlearning?scriptVersionId=134373208" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Implement Transfer learning with InceptionResNetV2-imagenet model**

In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load the pretrained model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freeze the base model
base_model.trainable = False

In [ ]:
# Add a global average pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
# Add a dense layer with 11 units and softmax activation for classification
output = tf.keras.layers.Dense(11, activation='softmax')(x)

In [ ]:
# Create the model
model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Create an image data generator for image augmentation
datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip=True,
fill_mode='nearest'
)

In [ ]:
# Create a training generator from the training folder
train_generator = datagen.flow_from_directory(
'/kaggle/input/food11-image-dataset/training',
target_size=(224, 224),
batch_size=32,
class_mode='categorical',
shuffle=True
)

# Create a validation generator from the validation folder
validation_generator = datagen.flow_from_directory(
'/kaggle/input/food11-image-dataset/validation',
target_size=(224, 224),
batch_size=32,
class_mode='categorical',
shuffle=False
)

# Create a evaluation generator from the evaluation folder
evaluation_generator = datagen.flow_from_directory(
'/kaggle/input/food11-image-dataset/evaluation',
target_size=(224, 224),
batch_size=32,
class_mode='categorical',
shuffle=False
)

In [ ]:
# Train the model for 10 epochs
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

In [ ]:
# Evaluate the model on the evaluation data using `evaluate`
print("Evaluate on evaluation data")
results = model.evaluate(evaluation_generator)
print("evaluation loss, evaluation acc:", results)

In [ ]:
# Import libraries for plotting
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Get the true and predicted labels from the evaluation data
y_true = evaluation_generator.classes
#y_pred = model.predict_classes(evaluation_generator)
y_pred = model.predict(evaluation_generator)
x=np.argmax(y_pred,axis=1)

# Plot the confusion matrix
plt.figure(figsize=(10, 10))
cm = confusion_matrix(y_true, x)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()